In [ ]:
import pandas as pd
import seaborn as sns
import os
import analysis


In [ ]:
analyser = analysis.Analyser("k6")
all_data = analyser.get_data()
all_data


In [ ]:
req_duration_data = all_data[all_data["metric_name"] == "http_req_duration"]
req_duration_data.columns


In [ ]:
ignore_vars = [
    "metric_name",
    "metric_value",
    "service",
    "subproto",
    "proto",
    "scenario",
    "status",
    "tls_version",
    "url",
    "name",
    "extra_tags",
    "metadata",
    "check",
    "error",
    "error_code",
    "expected_response",
    "group",
    "method",
]


In [ ]:
plot_data = req_duration_data.copy(deep=False)
analyser.plot_scatter(plot_data, ignore_vars=ignore_vars)


In [ ]:
plot_data = req_duration_data.copy(deep=False)
analyser.plot_ecdf(plot_data, ignore_vars=ignore_vars)
